In [1]:
import datetime as dt
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Import modules and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime, timedelta
import shrimpy
import psycopg2

In [2]:
class DatabaseClient:
    """
    Functionality for inserting and reading from the database
    """

    def __init__(self):
        # Connect to db
        self.con = psycopg2.connect(
            host='localhost',
            database='postgres',
            user='postgres',
            password='postgres'
        )
        self.article_table = pd.DataFrame()

    def read_db(self):
        cur = self.con.cursor()

        cur.execute("SELECT timestamp, comp_sentiment, sma, ema FROM sent_values WHERE timestamp > '2021-01-26 15:00:00'::timestamp;")

        rows = cur.fetchall()

        self.article_table = pd.DataFrame(data=rows, columns=['timestamp','compound','sma','ema'])


        return self.article_table

In [3]:
def get_latest_prices(trading_symbol):
    public_key = '12326758a39a720e15d064cab3c1f0a9332d107de453bd41926bb3acd565059e'
    secret_key = '6991cf4c9b518293429db0df6085d1731074bed8abccd7f0279a52fac5b0c1a8a2f6d28e11a50fbb1c6575d1407e637f9ad7c73fbddfa87c5d418fd58971f829'
    
    client = shrimpy.ShrimpyApiClient(public_key, secret_key)
        
    # get the candles for historical values
    candles = client.get_candles(
        'binance',  # exchange
        trading_symbol,      # base_trading_symbol
        'USDT',      # quote_trading_symbol
        '1h'       # interval
    )
        
    # Set the dataframe between these two dates
    tomorrows_date = (datetime.today() - timedelta(hours=1)).strftime("%Y-%m-%d %H:00")
    tomorrows_date = (datetime.today()).strftime("%Y-%m-%d %H:00")
    starting_date = '2021-01-25 02:00:00'

    # Put pulled cryptocurrency values into a dataframe and set dates
    prices_df = pd.DataFrame(candles)
    prices_df['time'] = pd.to_datetime(prices_df['time'], infer_datetime_format=True).dt.tz_localize(None)

    latest_prices = prices_df[(prices_df['time'] > starting_date) & (prices_df['time'] <tomorrows_date)]
    latest_prices['close'] = latest_prices['close'].astype('float64')

    return latest_prices

In [5]:
database_client = DatabaseClient()
df = database_client.read_db()
df.index = df.timestamp
df = df.iloc[:-1]

data = get_latest_prices('BTC')
data.index = data.time
data = data.iloc[:-4]

dataset_train = data.join(df, lsuffix=data.index, rsuffix=df.index)

dataset_train = dataset_train[['close', 'sma']]
dataset_train = dataset_train.dropna()
stk_data = dataset_train.astype('float')

stk_data

<ipython-input-3-b1d27c6bb073>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_prices['close'] = latest_prices['close'].astype('float64')


,close,sma
time,,
2021-02-07 21:00:00,38530.33,0.2386
2021-02-07 22:00:00,38797.02,0.2463
2021-02-07 23:00:00,38795.69,0.2555
2021-02-08 00:00:00,38725.40,0.2290
2021-02-08 01:00:00,38521.72,0.2290
...,...,...
2021-03-21 03:00:00,57790.00,-0.6158
2021-03-21 04:00:00,56909.83,-0.5980
2021-03-21 05:00:00,57129.99,-0.5980


In [86]:
#Data Preprocessing
stk_data['Date'] = stk_data.index
data2 = pd.DataFrame(columns = ['Date', 'Close'])
data2['Date'] = stk_data['Date']
data2['Close'] = stk_data['close']
data2['SMA'] = stk_data['sma']

train_set = data2.iloc[:850, 1:2].values
valid_set = data2.iloc[850:, 1:2].values

sc_train = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc_train.fit_transform(train_set)
X_train = []
y_train = []
for i in range(60, len(train_set)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0]) 
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

sc_valid = MinMaxScaler(feature_range = (0, 1))
valid_set_scaled = sc_valid.fit_transform(valid_set)
X_valid = []
y_valid = []
for i in range(60, len(valid_set)):
    X_valid.append(valid_set_scaled[i-60:i, 0])
    y_valid.append(valid_set_scaled[i, 0]) 
X_valid, y_valid = np.array(X_valid), np.array(y_valid)
X_valid = np.reshape(X_valid, (X_valid.shape[0], X_valid.shape[1], 1))

In [141]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 10))

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

#es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

tb = TensorBoard('logs')

regressor.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics=['mse'])
history = regressor.fit(X_train, y_train, epochs = 50, callbacks=[rlr, mcp, tb], validation_data=(X_valid, y_valid), batch_size = 1)

Epoch 1/50
790/790 [==============================] - 47s 43ms/step - loss: 0.0295 - mse: 0.0014 - val_loss: 0.1493 - val_mse: 0.0331

Epoch 00001: val_loss improved from inf to 0.14932, saving model to weights.h5
Epoch 2/50
790/790 [==============================] - 29s 36ms/step - loss: 0.0297 - mse: 0.0014 - val_loss: 0.1652 - val_mse: 0.0391

Epoch 00002: val_loss did not improve from 0.14932
Epoch 3/50
790/790 [==============================] - 28s 36ms/step - loss: 0.0312 - mse: 0.0016 - val_loss: 0.1178 - val_mse: 0.0225

Epoch 00003: val_loss improved from 0.14932 to 0.11775, saving model to weights.h5
Epoch 4/50
790/790 [==============================] - 28s 36ms/step - loss: 0.0279 - mse: 0.0013 - val_loss: 0.1297 - val_mse: 0.0261

Epoch 00004: val_loss did not improve from 0.11775
Epoch 5/50
790/790 [==============================] - 29s 36ms/step - loss: 0.0303 - mse: 0.0015 - val_loss: 0.1544 - val_mse: 0.0346

Epoch 00005: val_loss did not improve from 0.11775
Epoch 6/50

790/790 [==============================] - 30s 39ms/step - loss: 0.0216 - mse: 8.6430e-04 - val_loss: 0.1530 - val_mse: 0.0344

Epoch 00043: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00043: val_loss did not improve from 0.11775
Epoch 44/50
790/790 [==============================] - 30s 38ms/step - loss: 0.0218 - mse: 8.7933e-04 - val_loss: 0.1454 - val_mse: 0.0315

Epoch 00044: val_loss did not improve from 0.11775
Epoch 45/50
790/790 [==============================] - 30s 39ms/step - loss: 0.0226 - mse: 8.8671e-04 - val_loss: 0.1496 - val_mse: 0.0330

Epoch 00045: val_loss did not improve from 0.11775
Epoch 46/50
790/790 [==============================] - 31s 39ms/step - loss: 0.0223 - mse: 8.8340e-04 - val_loss: 0.1501 - val_mse: 0.0332

Epoch 00046: val_loss did not improve from 0.11775
Epoch 47/50
790/790 [==============================] - 31s 39ms/step - loss: 0.0217 - mse: 8.8083e-04 - val_loss: 0.1431 - val_mse: 0.0307

Epoch 00047: val_loss did no

In [ ]:
from matplotlib import pyplot

pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

In [ ]:
#predicting 246 values, using past 60 from the train data
inputs = data2.iloc[len(data2) - len(valid_set) - 60:,1:2].values
inputs = inputs.reshape(-inputs.shape[1],inputs.shape[1])
inputs  = sc_valid.transform(inputs)

In [ ]:
X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

In [ ]:
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
closing_price = regressor.predict(X_test)

# invert scaling for forecast
inv_yhat = np.concatenate((closing_price,X_test[:,-1]), axis=1)
closing_price = sc_valid.inverse_transform(inv_yhat)

In [ ]:
lstm_sma_rms=np.sqrt(np.mean(np.power((valid_set-closing_price),2)))
lstm_sma_rms

In [ ]:
#for plotting
train = data2[:850]
valid = data2[850:]
valid['Predictions'] = closing_price[:,0]
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])

plt.legend(labels=['Training Price', 'Actual Price', 'Predicted Price'])